<a href="https://colab.research.google.com/github/JasonTJH/mycolab/blob/main/TopNewFollowers_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Social Bakers (Top Twitter New Followers)

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import time
import sys
import re
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urljoin
from selenium import webdriver
import subprocess
from itertools import repeat, chain
from multiprocessing import Pool, Process, Manager
import requests
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from concurrent.futures import Future
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from collections import OrderedDict

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## Setup

In [ ]:
# Defining static & test variables
base = 'https://www.socialbakers.com'
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(base+'/statistics/twitter/profiles')
soup = BeautifulSoup(driver.page_source, "html.parser")
sample_link_ext = '/statistics/twitter/profiles/afghanistan'
sample_profile_ext = '/statistics/twitter/profiles/detail/111240404-radiomosaiquefm'
sample_chart_link = "https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=111240404&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months"
industry_list = ['/statistics/twitter/profiles', '/statistics/twitter/profiles/brands', '/statistics/twitter/profiles/celebrities', '/statistics/twitter/profiles/community', '/statistics/twitter/profiles/entertainment', '/statistics/twitter/profiles/media', '/statistics/twitter/profiles/place', '/statistics/twitter/profiles/society', '/statistics/twitter/profiles/sport']
sleep_time = 0.05 #for loading javascript in selenium parser - raise it if get_followers have no output

# Functions to be used below
def get_profile_ext(link_ext):
  my_list = list()
  driver.get(base + link_ext)
  soup = BeautifulSoup(driver.page_source, "lxml", parse_only=SoupStrainer(href=re.compile("/statistics/twitter/profiles/detail/")))
  outputs = [str(i) for i in soup]
  splitter = 'href="'
  splitter2 = '-'
  splitter3 = '"'
  for a in outputs:
    my_list.append(a.split(splitter)[1].split(splitter2)[0].split(splitter3)[0])
  return my_list
#print(get_profile_ext(sample_link_ext))


def get_chart_link(profile_ext):
  chart_link = 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids='+profile_ext.split('/detail/')[1].split('-')[0]+'&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months'
  return chart_link
#print(get_chart_link(sample_profile_ext))

def get_followers(chart_link):
  #driver = webdriver.Chrome('chromedriver',options=chrome_options)
  driver.get(chart_link)
  try:
    time.sleep(sleep_time)
    result = driver.execute_script('var text = options ; return text')
  except:
    try:
      time.sleep(sleep_time)
      result = driver.execute_script('var text = options ; return text')
    except:
      try:
        time.sleep(sleep_time)
        result = driver.execute_script('var text = options ; return text')
      except:
        return [], [], []
  # Cleaning data for use
  today = datetime(datetime.now().year, datetime.now().month, datetime.now().day)

  follower_list = []
  num_list = list(range(14*12,-14,-14))
  date_list = [today - timedelta(days= i) for i in num_list]
  for i,v in dict(dict(result)['series'][0])['data']:
    follower_list.append(v)
  return date_list, follower_list, num_list
#print(get_followers(sample_chart_link))

def plot_g(date_list, follower_list):
  plt.plot(date_list, follower_list)
#x, y, z = get_followers(sample_chart_link)
#plot_g(x,y)

def pct_chg(follower_list, num_list):
  try:
    followers_bf_2w = follower_list[num_list.index(14)] + 1 if follower_list[num_list.index(14)] == 0 else follower_list[num_list.index(14)]
    followers_bf_1m = follower_list[num_list.index(28)] + 1 if follower_list[num_list.index(28)] == 0 else follower_list[num_list.index(28)]
    followers_bf_3m = follower_list[num_list.index(84)] + 1 if follower_list[num_list.index(84)] == 0 else follower_list[num_list.index(84)]
    followers_bf_6m = follower_list[num_list.index(168)] + 1 if follower_list[num_list.index(168)] == 0 else follower_list[num_list.index(168)]
    followers_af = follower_list[-1]
    pct_chg_2w = round((followers_af/followers_bf_2w-1),5)
    pct_chg_1m = round((followers_af/followers_bf_1m-1),5)
    pct_chg_3m = round((followers_af/followers_bf_3m-1),5)
    pct_chg_6m = round((followers_af/followers_bf_6m-1),5)
    followers_bf_list = [followers_bf_6m, followers_bf_3m, followers_bf_1m, followers_bf_2w]
    pct_chg_list = [pct_chg_6m,pct_chg_3m,pct_chg_1m,pct_chg_2w]
  except:
    return
  return pct_chg_list, followers_bf_list, followers_af

#x, y, z = get_followers(sample_chart_link)
#print(pct_chg(y,z))

def get_category(profile_ext):
  #profile_ext = chart_link.split('?ids=')[1].split('&colors=')[0]
  driver.get(base + profile_ext)
  soup = BeautifulSoup(driver.page_source, "html.parser")
  category_set = set()
  for a in soup.find_all(href=True):
    if a['href'].startswith('/statistics/twitter/profiles/') and '/statistics/twitter/profiles/detail' not in a['href'] and a['href'] not in industry_list:
      category_set.add(str(a['href']).split('profiles/')[1])
  return category_set
#print(get_category(sample_profile_ext))

def get_profile_name(profile_ext):
  driver.get(base + profile_ext)
  soup = BeautifulSoup(driver.page_source, "html.parser")
  category_set = set()
  for a in soup.find_all('a', {'class' : 'external link-nw'}):
    profile_name = a.text
  return profile_name
#print(get_profile_name(sample_profile_ext))

def set_var(tenor,top_x):
  if tenor == '6m':
    target_list = sorted_chg_6m[len(sorted_chg_6m)-top_x:]
    target_value = 0
  elif tenor == '3m':
    target_list = sorted_chg_3m[len(sorted_chg_3m)-top_x:]
    target_value = 1
  elif tenor == '1m':
    target_list = sorted_chg_1m[len(sorted_chg_1m)-top_x:]
    target_value = 2
  elif tenor == '2w':
    target_list = sorted_chg_2w[len(sorted_chg_2w)-top_x:]
    target_value = 3
  else:
    pass
  return target_list, target_value, tenor, top_x
#print(set_var('2w',25)) # var1 takes in ['2w','1m','3m','6m'], var2 is the top # results you want to see - requires some output from code below before testing is allowed

### Getting Active List

In [ ]:
# Getting a list of directories to each countries page
active_list = ['/statistics/twitter/profiles/afghanistan', '/statistics/twitter/profiles/aland-islands', '/statistics/twitter/profiles/albania', '/statistics/twitter/profiles/algeria', '/statistics/twitter/profiles/american-samoa', '/statistics/twitter/profiles/andorra', '/statistics/twitter/profiles/angola', '/statistics/twitter/profiles/anguilla', '/statistics/twitter/profiles/antigua-and-barbuda', '/statistics/twitter/profiles/argentina', '/statistics/twitter/profiles/armenia', '/statistics/twitter/profiles/aruba', '/statistics/twitter/profiles/australia', '/statistics/twitter/profiles/austria', '/statistics/twitter/profiles/azerbaijan', '/statistics/twitter/profiles/bahamas', '/statistics/twitter/profiles/bahrain', '/statistics/twitter/profiles/bangladesh', '/statistics/twitter/profiles/barbados', '/statistics/twitter/profiles/belarus', '/statistics/twitter/profiles/belgium', '/statistics/twitter/profiles/belize', '/statistics/twitter/profiles/benin', '/statistics/twitter/profiles/bermuda', '/statistics/twitter/profiles/bhutan', '/statistics/twitter/profiles/bolivia', '/statistics/twitter/profiles/bonaire-saint-eustatius-and-saba', '/statistics/twitter/profiles/bosnia-and-herzegovina', '/statistics/twitter/profiles/botswana', '/statistics/twitter/profiles/brazil', '/statistics/twitter/profiles/british-indian-ocean-territory', '/statistics/twitter/profiles/british-virgin-islands', '/statistics/twitter/profiles/brunei', '/statistics/twitter/profiles/bulgaria', '/statistics/twitter/profiles/burkina-faso', '/statistics/twitter/profiles/burundi', '/statistics/twitter/profiles/cambodia', '/statistics/twitter/profiles/cameroon', '/statistics/twitter/profiles/canada', '/statistics/twitter/profiles/cape-verde', '/statistics/twitter/profiles/cayman-islands', '/statistics/twitter/profiles/central-african-republic', '/statistics/twitter/profiles/chad', '/statistics/twitter/profiles/chile', '/statistics/twitter/profiles/china', '/statistics/twitter/profiles/cocos-islands', '/statistics/twitter/profiles/colombia', '/statistics/twitter/profiles/comoros', '/statistics/twitter/profiles/costa-rica', '/statistics/twitter/profiles/croatia', '/statistics/twitter/profiles/cuba', '/statistics/twitter/profiles/curacao', '/statistics/twitter/profiles/cyprus', '/statistics/twitter/profiles/czech-republic', '/statistics/twitter/profiles/democratic-republic-of-the-congo', '/statistics/twitter/profiles/denmark', '/statistics/twitter/profiles/djibouti', '/statistics/twitter/profiles/dominica', '/statistics/twitter/profiles/dominican-republic', '/statistics/twitter/profiles/east-timor', '/statistics/twitter/profiles/ecuador', '/statistics/twitter/profiles/egypt', '/statistics/twitter/profiles/el-salvador', '/statistics/twitter/profiles/equatorial-guinea', '/statistics/twitter/profiles/eritrea', '/statistics/twitter/profiles/estonia', '/statistics/twitter/profiles/ethiopia', '/statistics/twitter/profiles/falkland-islands', '/statistics/twitter/profiles/faroe-islands', '/statistics/twitter/profiles/fiji', '/statistics/twitter/profiles/finland', '/statistics/twitter/profiles/france', '/statistics/twitter/profiles/french-guiana', '/statistics/twitter/profiles/french-polynesia', '/statistics/twitter/profiles/global', '/statistics/twitter/profiles/gabon', '/statistics/twitter/profiles/gambia', '/statistics/twitter/profiles/georgia', '/statistics/twitter/profiles/germany', '/statistics/twitter/profiles/ghana', '/statistics/twitter/profiles/gibraltar', '/statistics/twitter/profiles/greece', '/statistics/twitter/profiles/greenland', '/statistics/twitter/profiles/grenada', '/statistics/twitter/profiles/guadeloupe', '/statistics/twitter/profiles/guam', '/statistics/twitter/profiles/guatemala', '/statistics/twitter/profiles/guernsey', '/statistics/twitter/profiles/guinea', '/statistics/twitter/profiles/guinea-bissau', '/statistics/twitter/profiles/guyana', '/statistics/twitter/profiles/haiti', '/statistics/twitter/profiles/honduras', '/statistics/twitter/profiles/hong-kong', '/statistics/twitter/profiles/hungary', '/statistics/twitter/profiles/iceland', '/statistics/twitter/profiles/india', '/statistics/twitter/profiles/indonesia', '/statistics/twitter/profiles/iran', '/statistics/twitter/profiles/iraq', '/statistics/twitter/profiles/ireland', '/statistics/twitter/profiles/isle-of-man', '/statistics/twitter/profiles/israel', '/statistics/twitter/profiles/italy', '/statistics/twitter/profiles/ivory-coast', '/statistics/twitter/profiles/jamaica', '/statistics/twitter/profiles/japan', '/statistics/twitter/profiles/jersey', '/statistics/twitter/profiles/jordan', '/statistics/twitter/profiles/kazakhstan', '/statistics/twitter/profiles/kenya', '/statistics/twitter/profiles/kosovo', '/statistics/twitter/profiles/kuwait', '/statistics/twitter/profiles/kyrgyzstan', '/statistics/twitter/profiles/laos', '/statistics/twitter/profiles/latvia', '/statistics/twitter/profiles/lebanon', '/statistics/twitter/profiles/lesotho', '/statistics/twitter/profiles/liberia', '/statistics/twitter/profiles/libya', '/statistics/twitter/profiles/liechtenstein', '/statistics/twitter/profiles/lithuania', '/statistics/twitter/profiles/luxembourg', '/statistics/twitter/profiles/macao', '/statistics/twitter/profiles/macedonia', '/statistics/twitter/profiles/madagascar', '/statistics/twitter/profiles/malawi', '/statistics/twitter/profiles/malaysia', '/statistics/twitter/profiles/maldives', '/statistics/twitter/profiles/mali', '/statistics/twitter/profiles/malta', '/statistics/twitter/profiles/marshall-islands', '/statistics/twitter/profiles/martinique', '/statistics/twitter/profiles/mauritania', '/statistics/twitter/profiles/mauritius', '/statistics/twitter/profiles/mayotte', '/statistics/twitter/profiles/mexico', '/statistics/twitter/profiles/micronesia', '/statistics/twitter/profiles/moldova', '/statistics/twitter/profiles/monaco', '/statistics/twitter/profiles/mongolia', '/statistics/twitter/profiles/montenegro', '/statistics/twitter/profiles/montserrat', '/statistics/twitter/profiles/morocco', '/statistics/twitter/profiles/mozambique', '/statistics/twitter/profiles/myanmar', '/statistics/twitter/profiles/namibia', '/statistics/twitter/profiles/nauru', '/statistics/twitter/profiles/nepal', '/statistics/twitter/profiles/netherlands', '/statistics/twitter/profiles/netherlands-antilles', '/statistics/twitter/profiles/new-caledonia', '/statistics/twitter/profiles/new-zealand', '/statistics/twitter/profiles/nicaragua', '/statistics/twitter/profiles/niger', '/statistics/twitter/profiles/nigeria', '/statistics/twitter/profiles/norfolk-island', '/statistics/twitter/profiles/northern-mariana-islands', '/statistics/twitter/profiles/norway', '/statistics/twitter/profiles/oman', '/statistics/twitter/profiles/pakistan', '/statistics/twitter/profiles/palau', '/statistics/twitter/profiles/palestinian-territory', '/statistics/twitter/profiles/panama', '/statistics/twitter/profiles/papua-new-guinea', '/statistics/twitter/profiles/paraguay', '/statistics/twitter/profiles/peru', '/statistics/twitter/profiles/philippines', '/statistics/twitter/profiles/poland', '/statistics/twitter/profiles/portugal', '/statistics/twitter/profiles/puerto-rico', '/statistics/twitter/profiles/qatar', '/statistics/twitter/profiles/republic-of-the-congo', '/statistics/twitter/profiles/reunion', '/statistics/twitter/profiles/romania', '/statistics/twitter/profiles/russia', '/statistics/twitter/profiles/rwanda', '/statistics/twitter/profiles/saint-kitts-and-nevis', '/statistics/twitter/profiles/saint-lucia', '/statistics/twitter/profiles/saint-martin', '/statistics/twitter/profiles/saint-vincent-and-the-grenadines', '/statistics/twitter/profiles/samoa', '/statistics/twitter/profiles/san-marino', '/statistics/twitter/profiles/sao-tome-and-principe', '/statistics/twitter/profiles/saudi-arabia', '/statistics/twitter/profiles/senegal', '/statistics/twitter/profiles/serbia', '/statistics/twitter/profiles/seychelles', '/statistics/twitter/profiles/sierra-leone', '/statistics/twitter/profiles/singapore', '/statistics/twitter/profiles/slovakia', '/statistics/twitter/profiles/slovenia', '/statistics/twitter/profiles/somalia', '/statistics/twitter/profiles/south-africa', '/statistics/twitter/profiles/south-korea', '/statistics/twitter/profiles/south-sudan', '/statistics/twitter/profiles/spain', '/statistics/twitter/profiles/sri-lanka', '/statistics/twitter/profiles/sudan', '/statistics/twitter/profiles/suriname', '/statistics/twitter/profiles/swaziland', '/statistics/twitter/profiles/sweden', '/statistics/twitter/profiles/switzerland', '/statistics/twitter/profiles/syria', '/statistics/twitter/profiles/taiwan', '/statistics/twitter/profiles/tajikistan', '/statistics/twitter/profiles/tanzania', '/statistics/twitter/profiles/thailand', '/statistics/twitter/profiles/togo', '/statistics/twitter/profiles/tokelau', '/statistics/twitter/profiles/tonga', '/statistics/twitter/profiles/trinidad-and-tobago', '/statistics/twitter/profiles/tunisia', '/statistics/twitter/profiles/turkey', '/statistics/twitter/profiles/turkmenistan', '/statistics/twitter/profiles/turks-and-caicos-islands', '/statistics/twitter/profiles/u-s-virgin-islands', '/statistics/twitter/profiles/uganda', '/statistics/twitter/profiles/ukraine', '/statistics/twitter/profiles/united-arab-emirates', '/statistics/twitter/profiles/united-kingdom', '/statistics/twitter/profiles/united-states', '/statistics/twitter/profiles/uruguay', '/statistics/twitter/profiles/uzbekistan', '/statistics/twitter/profiles/vanuatu', '/statistics/twitter/profiles/vatican', '/statistics/twitter/profiles/venezuela', '/statistics/twitter/profiles/vietnam', '/statistics/twitter/profiles/yemen', '/statistics/twitter/profiles/zambia', '/statistics/twitter/profiles/zimbabwe']
print(len(active_list),active_list)

### Getting Profile Ext List (5 mins)

In [ ]:
# Scours each countries' page to get the top profiles and put them in a list
profile_ext_list = []
start = time.time()
count = 0
for i in active_list:
  profile_ext_list.append(get_profile_ext(i))
  if count % 10 == 0:
    print(count,'out of',len(active_list),' countries completed.')
  count+=1
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
profile_ext_list = list(set(chain.from_iterable(profile_ext_list)))
print(len(profile_ext_list),profile_ext_list)

### Getting Static Profile Ext List [Start here if you recently ran (and saved output for) above code]

In [ ]:
# Done in around 5 mins
static_profile_ext_list = [] # Save output from the previous here to save time

try:
  profile_ext_list = profile_ext_list
except:
  profile_ext_list = static_profile_ext_list

### Getting Chart Link List

In [ ]:
# Obtains chart links from the profile links
chart_link_list = []
start = time.time()
for i in profile_ext_list:
  chart_link_list.append(get_chart_link(i))
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
chart_link_list = list(set(chart_link_list))
print(len(chart_link_list),chart_link_list)

### Get User-Follower List (20 mins)

In [ ]:
# Scraps the followers from the list of profiles from social bakers website - Default condition set to obtain profiles with more than 10,000 followers and have consistent follower growth over (2w,1m,3m,6m)
driver = webdriver.Chrome('chromedriver',options=chrome_options)
output = dict()
count = 1
start = time.time()

for i in chart_link_list:
  profile_ext = i.split('?ids=')[1].split('&colors=')[0]
  try:
    _, follower_list, num_list = get_followers(i)
    if follower_list[-1] >= 10000 and follower_list[6]>follower_list[0] and follower_list[10]>follower_list[6] and follower_list[11]>follower_list[10]: # amend conditions here
      pct_chg_list, followers_bf_list, followers_af = pct_chg(follower_list, num_list)
      pct_chg_list.append(followers_af)
      output[profile_ext] = pct_chg_list
      #print(profile_ext, str(round(output[profile_ext][0]*100,2)) + '% growth over 6m, currently has ' + str(followers_af) + ' followers')
  except:
    pass
    #print("NO DATA")
  if count % 100 == 0:
    print(str(count)+" out of " + str(len(chart_link_list)) + " completed.", len(output),"inserted")
  count += 1      
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
print(len(output),output)

### Getting Static User-Follower List [Start here if you recently ran (and saved output for) above code]

In [ ]:
#done in 20 mins
static_output = {}

try:
  output = output
except:
  output = static_output

sorted_chg_6m = sorted([float(i[0]) for i in [i for i in output.values()]])
sorted_chg_3m = sorted([float(i[1]) for i in [i for i in output.values()]])
sorted_chg_1m = sorted([float(i[2]) for i in [i for i in output.values()]])
sorted_chg_2w = sorted([float(i[3]) for i in [i for i in output.values()]])

### Get top new follower % accounts

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
#Setting targets
target_list, target_value, tenor, top_x = set_var('2w',25) # set tenor and top # of profiles to view
target_count = 1
mydict = OrderedDict()

for i in target_list[::-1]:
  for key, value in output.items():
    if value[target_value] == i:# and int(value[0]+1) != value[4]: #To remove division by 0 followers data
      profile_ext = '/statistics/twitter/profiles/detail/' + key
      profile_name = get_profile_name(profile_ext)
      category = get_category(profile_ext)
      print(target_count,tenor+' chg = '+str(round(value[target_value]*100,2))+'%',profile_name, category, base+profile_ext, value)                                    
      mydict[profile_name] = {'URL' : base+profile_ext,
                              'Category' : category,
                              'Followers' : f"{value[4]:,d}",
                              '6m chg' : f"{value[0]:.0%}",#str(int(value[0]*100)) +'%',
                              '3m chg' : f"{value[1]:.0%}",
                              '1m chg' : f"{value[2]:.0%}",
                              '0.5m chg' : f"{value[3]:.0%}"}
  target_count+=1

1 2w chg = 45.4% @ONPE_oficial {'society/governmental', 'peru'} https://www.socialbakers.com/statistics/twitter/profiles/detail/255707941 [0.9937, 0.91543, 0.4596, 0.45404, 525490]
2 2w chg = 38.87% @intel_la {'global', 'brands/electronics'} https://www.socialbakers.com/statistics/twitter/profiles/detail/53459457 [0.44089, 0.41333, 0.38911, 0.38869, 91855]
3 2w chg = 29.03% @taoyuanairport {'taiwan', 'place/airport'} https://www.socialbakers.com/statistics/twitter/profiles/detail/3306946201 [0.97065, 0.90688, 0.29522, 0.29028, 19269]
4 2w chg = 26.56% @WePlay_Esports {'ukraine'} https://www.socialbakers.com/statistics/twitter/profiles/detail/949934119 [0.51783, 0.46346, 0.40693, 0.26562, 29575]
5 2w chg = 23.14% @TEAMEVGA {'brands/services', 'united-states', 'united-kingdom'} https://www.socialbakers.com/statistics/twitter/profiles/detail/26180582 [0.26966, 0.26344, 0.24535, 0.23136, 631722]
6 2w chg = 18.29% @carlitosrejala {'society/politics', 'paraguay'} https://www.socialbakers.com

In [ ]:
mydict

OrderedDict([('@ONPE_oficial',
              {'0.5m chg': '45%',
               '1m chg': '46%',
               '3m chg': '92%',
               '6m chg': '99%',
               'Category': {'peru', 'society/governmental'},
               'Followers': '525,490',
               'URL': 'https://www.socialbakers.com/statistics/twitter/profiles/detail/255707941'}),
             ('@intel_la',
              {'0.5m chg': '39%',
               '1m chg': '39%',
               '3m chg': '41%',
               '6m chg': '44%',
               'Category': {'brands/electronics', 'global'},
               'Followers': '91,855',
               'URL': 'https://www.socialbakers.com/statistics/twitter/profiles/detail/53459457'}),
             ('@taoyuanairport',
              {'0.5m chg': '29%',
               '1m chg': '30%',
               '3m chg': '91%',
               '6m chg': '97%',
               'Category': {'place/airport', 'taiwan'},
               'Followers': '19,269',
               'URL': 'ht